In [1]:
import random
import math
from functools import lru_cache

In [2]:
import metaheuristics as mh

In [3]:
class Michalewicz(mh.abstract.Problem):
    
    def __init__(self, d, m, *args, **kwargs):
        self.d = d
        self.m = m
        super().__init__(*args, **kwargs)
    
    def config(self):
        pass

    @property
    def random_solution(self):
        return Solution(self)

    def solution(self, *args, **kwargs):
        return Solution(self, *args, **kwargs)
    
    def reset(self):
        pass
    
    @property
    def difficulty(self):
        return self.d
    
    def __hash__(self):
        return self.m

In [4]:
class Solution(mh.utils.BaseSolutionMixin, mh.abstract.Solution):
    
    def __init__(self, problem, sequence=None):
        super().__init__(problem, sequence=sequence)

    @property
    def sequence(self):
        if not self.__sequence:
            self.__sequence = [random.uniform(0, math.pi) for i in range(self.problem.d)]
        return self.__sequence

    @sequence.setter
    def sequence(self, value):
        self.__sequence = value
        self.__cost = None
        
    @property
    def is_complete(self):
        return not self.is_partial

    @property
    def is_partial(self):
        return False
    
    def _correct(self):
        return True
    
    @property
    def correct(self):
        return self._correct()
    
    @lru_cache(maxsize=2 ** 20)
    def _cost(self):
        return -sum(
            math.sin(x) * (math.sin(((i+1) * x**2)/math.pi))**(2*self.problem.m)
            for i, x in enumerate(self.sequence))
    
    @property
    def cost(self):
        return self._cost()

In [5]:
problem = Michalewicz(d=5, m=5)

In [6]:
ts = mh.algorithms.TabuSearch(
    problem,
    n_iters=5000,
    search_operator=mh.utils.tweak(2, 5, 0, .5)
)
ts()
ts.best_solution

[2.185516171994295, 1.5707811603469786, 1.2864213379873104, 1.9223310432847303, 1.721391065985426] - -4.69

In [7]:
res = []

for _ in range(10):
    ts = mh.algorithms.TabuSearch(
        problem,
        n_iters=5000,
        search_operator=mh.utils.tweak(2, 5, 0, .5)
    )
    ts()
    res.append(ts.best_solution.cost)

In [8]:
res

[-4.693331446249454,
 -4.693404247541943,
 -4.693257644606561,
 -4.691844638041525,
 -4.69343292266179,
 -4.69341210830257,
 -4.693380184090866,
 -4.693271700752772,
 -4.69335247842594,
 -4.692777060811082]

In [9]:
sa = mh.algorithms.SimulatedAnnealing(
    problem,
    n_iters=5000,
    search_operator=mh.utils.tweak(2, 5, 0, .5)
)
sa()
sa.best_solution

[2.198114511961814, 7.5373770586749105, 1.2864913499192872, 1.108158367379031, 1.719821936025526] - -4.59

In [10]:
res = []

for _ in range(10):
    sa = mh.algorithms.SimulatedAnnealing(
        problem,
        n_iters=5000,
        search_operator=mh.utils.tweak(2, 5, 0, .5)
    )
    sa()
    res.append(sa.best_solution.cost)

In [11]:
res

[-4.6703445499540335,
 -4.633974611789599,
 -4.59771781715317,
 -4.639219490861327,
 -4.658755389849041,
 -4.606920505583271,
 -4.78065610128027,
 -4.570038757248499,
 -4.599668445552558,
 -4.632864733568315]

In [12]:
ga = mh.algorithms.Genetic(
    problem,
    pop_size=50,
    n_iters=5000,
    crossover=mh.utils.continuous_order_crossover,
    prob_mutation=0.1,
    search_operator=mh.utils.tweak(2, 5, 0, .5)
)
ga()
ga.best_solution

[2.1984392576999188, 1.5635791373212444, 2.248297103478114, 1.901653510209611, 1.7345621003331324] - -4.38

In [13]:
res = []

for _ in range(10):
    ga = mh.algorithms.Genetic(
        problem,
        pop_size=50,
        n_iters=10_000,
        crossover=mh.utils.continuous_order_crossover,
        prob_mutation=0.1,
        search_operator=mh.utils.tweak(2, 5, 0, .5)
    )
    ga()
    res.append(ga.best_solution.cost)

In [14]:
res

[-4.085004709306434,
 -4.051410449372484,
 -4.019196616558486,
 -4.323702010794092,
 -3.99276341752074,
 -4.026708212363287,
 -4.119574118685985,
 -4.264054277719343,
 -4.377642830352868,
 -4.3155804591740905]